# RAG
1. Load the VecotrDB from pre-processed data
2. Setup LLM
3. Setup Conversional Retirever

## Open items
1. LLM is very slow
2. How to leverge LLM to formulate a vectorDB search using filters

In [ ]:
#from huggingface_hub import notebook_login
#notebook_login()
#
# from huggingface_hub import login
# login(token="FILL TOKEN")

In [3]:
# import warnings
# warnings.filterwarnings('ignore')

# from google.colab import drive
# drive.mount('/content/drive')
#%cd "/gdrive/MyDrive/Interview Kickstart/MLSwitchup/Capstone/ShopTalk"
# %cd /content/drive/MyDrive/ik-ml/capstone
# %ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/ik-ml/capstone
 11-27-brahm-ShopTalkEDA_local.ipynb   old/
 ABO_dataset/                          openai-api-key.gdoc
 chroma/                               rag-imges.ipynb
 chroma_langchain_db/                  st-embeddings-and-vector-db.ipynb
 data-old/                             st-pretrained-llm-with-vecotrdb-search.ipynb
 github/                               st-vecotr-db.ipynb
'higgingface-access-token=.gdoc'       st-vector-db-02.ipynb
 llama32/


In [4]:
# !pip install langchain_community
# !pip install chromadb
# Ref: https://python.langchain.com/docs/integrations/vectorstores/chroma/
# !pip install -qU langchain-huggingface
# !pip install -qU "langchain-chroma>=0.1.2"

# STEP 01: Load the vectorDB from file (preprocessed in earlier step)

In [1]:
# import chromadb.utils.embedding_functions as embedding_functions
from langchain_chroma import Chroma

# from langchain_huggingface import HuggingFaceEmbeddings


In [6]:
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [2]:
vector_store = Chroma(
    collection_name="st_gs",
    # embedding_function=embeddings,
    persist_directory="/mnt/d/db",  # Where to save data locally, remove if not necessary
)

In [3]:
retriever = vector_store.as_retriever()

In [ ]:
# retriever.invoke("Give me all the products whose brand is Whole Foods?")

ValueError: You must provide an embedding function to compute embeddings.https://docs.trychroma.com/guides/embeddings in query.

# STEP-02:  Setup LLM

In [10]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

#model_id = "meta-llama/Llama-3.2-11B-Vision-Instruct"
model_id = "meta-llama/Llama-3.2-3B-Instruct"

# Load the tokenizer and model
print('Getting Tokenizer')
tokenizer = AutoTokenizer.from_pretrained(model_id, user_auth_token=True)
print('Getting Model ...')
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,  # Specify dtype here
    device_map="auto",  # Specify device map here
    # user_auth_token=True,
)

Getting Tokenizer


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Getting Model ...


...

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

# STEP-03: Setup Conversational Retrieval Chain

Conversational Retrieval Chain allows us to create chatbots that can answer follow up questions. This requires that the LLM has knowledge of the history of the conversation. LangChain provides us with Conversational Retrieval Chain that works not just on the recent input, but the whole chat history

## 2.1 Prompt To Generate Search Query For Retriever
The prompt contains the user input, the chat history, and a message to generate a search query.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder

prompt_search_query = ChatPromptTemplate.from_messages([
MessagesPlaceholder(variable_name="chat_history"),
("user","{input}"),
("user","Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])

## 2.2 Retriever Chain
We use the create_history_aware_retriever chain to retrieve the relevant data from the vector store.

The create_history_aware_retriever creates a chain that does the following steps:

Sends a prompt to the LLM with the chat_history and user input to generate a search query for the retriever
The retriever uses the search query to obtain the relevant documents from the vector store.
So, the inputs to the create_history_aware_retriever are the llm, retriever and the prompt.

The retriever_chain will return the retrieved documents from the vector store that are relevant to the user input and chat history.

In [ ]:
from langchain_community.llms import HuggingFacePipeline

pipe = pipeline(
    "text-generation", model=model, tokenizer=tokenizer,
    max_new_tokens=512,
    max_length=1024
)
hf = HuggingFacePipeline(pipeline=pipe)

<ipython-input-11-2b6178bcdff4>:8: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  hf = HuggingFacePipeline(pipeline=pipe)


In [ ]:
from langchain.chains import create_history_aware_retriever

In [ ]:
retriever_chain = create_history_aware_retriever(hf, retriever, prompt_search_query)

## 2.3 Prompt To Get Response From LLM Based on Chat History
The next step is to send the retrieved documents from the vector store along with a prompt to the llm to get the response to the user input.

We create a prompt containing the context (retrieved documents from vector store), chat history and the user input.

In [ ]:
prompt_get_answer = ChatPromptTemplate.from_messages(
    [
("system", "Answer the user's questions based on the below context:\\n\\n{context}"),
MessagesPlaceholder(variable_name="chat_history"),("user","{input}")
])

## 2.4 Document Chain
Next, we create a chain using create_stuff_documents_chain which will send the prompt to the llm.

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain=create_stuff_documents_chain(hf,prompt_get_answer)

## 2.5 Conversational Retrieval Chain
So, in the final step, we combine retriever_chain and document_chain using create_retrieval_chain to create a Conversational retrieval chain.

In [ ]:
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

To recap, we now have a retriever_chain that retrieves the relevant data from vector store, and document_chain that sends the chat_history, relevant data and user input to the llm.

## 2.6 Invoking the Chain
All that is remaining is to invoke it.

To test it, we create a sample chat_history and then invoke the retrieval_chain.

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Give me all the products whose weight is 5 or more?"), AIMessage(content="Yes")]

response = retrieval_chain.invoke({
"chat_history":chat_history,
"input":"Get the item names and item color and item weight?"
})
print (response['answer'])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/

System: Answer the user's questions based on the below context:\n\n[] brand : Whole Foods Market bullet_point : ['Brought to you by Whole Foods Market', 'Made in France', 'Made with cage-free eggs', 'Vegetarian'] color : nan fabric_type : nan item_name : Whole Foods Market Brioche Loaf With Chocolate Swirl, 14.11 Oz item_weight : 14.11 model_name : nan product_type : BREAD style : nan main_image_id : 51RLRY9k-+L other_image_id : ['41el7cvdjGL', '51V1i85l-fL', '61K5ijmXizL', '71psf7RBjrL'] country : US marketplace : Amazon domain_name : amazon.com material : nan item_keywords : ['brioche', 'brioche loaf', 'chocolate', 'chocolate swirl', 'france', 'cage-free eggs', 'vegetarian', 'snack', 'breakfast', 'dessert', 'coffee', 'whole foods', 'wfm', 'whole foods market', 'treat', 'french', 'bakery'] pattern : nan model_year : nan finish_type : nan

[] brand : Allegro Coffee bullet_point : ['Brought to you by Whole Foods Market', 'Specialty whole bean coffee', 'Light roast', 'Herbal, fragrant, g

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Give me all the products whose weight is 1.25 or more?"), AIMessage(content="Yes")]

response = retrieval_chain.invoke({
"chat_history":chat_history,
"input":"How?"
})
print (response['answer'])

NameError: name 'retrieval_chain' is not defined